In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import scipy as sp
import string
import json
import re
from sklearn.utils import shuffle
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as  plt
import seaborn as sns
import sklearn
from sklearn.metrics import confusion_matrix
####
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn import linear_model
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection

from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

In [2]:
# open training dataset and parse recipes

with open('train.json') as cooking_file:  
    
    data = json.load(cooking_file)
    

In [3]:
#COUNTVECTORIZER
#removing punctuations and spaces before fixing the data and keep them all in a dict  (italian, mexican)

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 

            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.replace(' ', '').lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list


clean_italian_cuisine_data = gather_recipes(data, 'italian')
clean_mexican_cuisine_data = gather_recipes(data, 'mexican')

clean_filipino_cuisine_data = gather_recipes(data, 'filipino')
clean_british_cuisine_data = gather_recipes(data, 'british')
clean_moroccan_cuisine_data = gather_recipes(data, 'moroccan')

In [4]:
#creating dfs  (italian, mexican)

italian_df = DataFrame(clean_italian_cuisine_data)

mexican_df = DataFrame(clean_mexican_cuisine_data)


#creating dfs (filipino, british, moroccan )

filipino_df = DataFrame(clean_filipino_cuisine_data) 

british_df = DataFrame(clean_british_cuisine_data)

moroccan_df = DataFrame(clean_moroccan_cuisine_data)


In [5]:
#fixing data for CountVectorizer (italian, mexican)

it_unique_ids = []
it_ingredients = []
it_grouped = italian_df.groupby('id')

for ids, ing in it_grouped:
    
    it_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    it_ingredients.append(row)

    
mex_unique_ids = []
mex_ingredients = []
mex_grouped = mexican_df.groupby('id')

for ids, ing in mex_grouped:
    
    mex_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    mex_ingredients.append(row)
    
#fixing data for CountVectorizer (moroccan, filipino, british)

moroccan_unique_ids = []
moroccan_ingredients = []

moroccan_grouped = moroccan_df.groupby('id')

for ids, ing in moroccan_grouped:
    
    moroccan_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    moroccan_ingredients.append(row)


filipino_unique_ids = []
filipino_ingredients = []

filipino_grouped = filipino_df.groupby('id')

for ids, ing in filipino_grouped:
    
    filipino_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    filipino_ingredients.append(row)


british_unique_ids = []
british_ingredients = []

british_grouped = british_df.groupby('id')

for ids, ing in british_grouped:
    
    british_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    british_ingredients.append(row)

In [6]:
#using CountVectorizer  (italian, mexican) and adding one column Cuisine

it_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
it_array = it_vectorizer.fit_transform(it_ingredients)
it_array = it_array.toarray()
it_df = DataFrame(it_array, columns = it_vectorizer.get_feature_names(), index = it_unique_ids)
it_df['Cuisine'] = 0 #italian
it_df.to_csv('it_cleandata.csv', index_label = 'ID')

mex_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex_array = mex_vectorizer.fit_transform(mex_ingredients)
mex_array = mex_array.toarray()
mex_df = DataFrame(mex_array, columns = mex_vectorizer.get_feature_names(), index = mex_unique_ids)
mex_df['Cuisine'] = 1 #mexican
mex_df.to_csv('mex_cleandata.csv', index_label = 'ID')

#using CountVectorizer (moroccan, filipino, british)

moroccan_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
moroccan_array = moroccan_vectorizer.fit_transform(moroccan_ingredients)
moroccan_array = moroccan_array.toarray()
moroccan_df = DataFrame(moroccan_array, columns = moroccan_vectorizer.get_feature_names(), index = moroccan_unique_ids)
moroccan_df['Cuisine'] = 0 #moroccan
moroccan_df.to_csv('moroccan_cleandata.csv', index_label = 'ID')

filipino_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
filipino_array = filipino_vectorizer.fit_transform(filipino_ingredients)
filipino_array = filipino_array.toarray()
filipino_df = DataFrame(filipino_array, columns = filipino_vectorizer.get_feature_names(), index = filipino_unique_ids)
filipino_df['Cuisine'] = 1 #filipino
filipino_df.to_csv('filipino_cleandata.csv', index_label = 'ID')

british_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
british_array = british_vectorizer.fit_transform(british_ingredients)
british_array = british_array.toarray()
british_df = DataFrame(british_array, columns = british_vectorizer.get_feature_names(), index = british_unique_ids)
british_df['Cuisine'] = 2 #british
british_df.to_csv('british_cleandata.csv', index_label = 'ID')

In [7]:
#putting mexican and italian in one DataFrame and shuffle

df = pd.DataFrame(it_df)
df = df.append(mex_df, sort = False)
df.fillna(0, inplace = True)
df = df.astype('int64')

df = shuffle(df)

#putting moroccan, filipino, british in one DataFrame and shuffle

df_ = pd.DataFrame(moroccan_df)
df_ = df_.append(filipino_df, sort = False)
df_ = df_.append(british_df, sort = False)
df_.fillna(0, inplace = True)
df_ = df_.astype('int64')

df_ = shuffle(df_)

In [8]:
#TFIDFVECTORIZER
#removing punctuations before fixing the data and keep them all in a dict  (italian, mexican)

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 
            
            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list

clean2_italian_cuisine_data = gather_recipes(data, 'italian')
clean2_mexican_cuisine_data = gather_recipes(data, 'mexican')

In [9]:
#creating dfs2  (italian, mexican)

italian2_df = DataFrame(clean2_italian_cuisine_data)

mexican2_df = DataFrame(clean2_mexican_cuisine_data)

In [10]:
#fixing data for TfidfVectorizer (italian, mexican)

it2_unique_ids = []
it2_ingredients = []
it2_grouped = italian2_df.groupby('id')

for ids, ing in it2_grouped:
    
    it2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    it2_ingredients.append(row)

    
mex2_unique_ids = []
mex2_ingredients = []
mex2_grouped = mexican2_df.groupby('id')

for ids, ing in mex2_grouped:
    
    mex2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    mex2_ingredients.append(row)


In [32]:
#using TfidfVectorizer  (italian, mexican) and adding one column Cuisine

it2_vectorizer = TfidfVectorizer(analyzer = 'word', binary = True)
it2_array = it2_vectorizer.fit_transform(it2_ingredients)
it2_array = it2_array.toarray()
it2_df = DataFrame(it2_array, columns = it2_vectorizer.get_feature_names(), index = it2_unique_ids)
it2_df['Cuisine'] = 0 #italian
it2_df.to_csv('it2_cleandata.csv', index_label = 'ID')

mex2_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex2_array = mex2_vectorizer.fit_transform(mex2_ingredients)
mex2_array = mex2_array.toarray()
mex2_df = DataFrame(mex2_array, columns = mex2_vectorizer.get_feature_names(), index = mex2_unique_ids)
mex2_df['Cuisine'] = 1 #mexican
mex2_df.to_csv('mex2_cleandata.csv', index_label = 'ID')

In [33]:
#putting mexican and italian in one DataFrame and shuffle

df2 = pd.DataFrame(it2_df)
df2 = df2.append(mex2_df, sort = False)
df2.fillna(0, inplace = True)

df2 = shuffle(df2)

In [13]:
# Multiple Logistic Regression 
lr_m = linear_model.LogisticRegression()

# Support Vector Machines
svm_m = svm.LinearSVC(random_state=0)
#Decision Trees
dtree = tree.DecisionTreeClassifier()

#k-NN
knn = KNeighborsClassifier(n_neighbors = 3)

# Naive Bayes
gnb = BernoulliNB(binarize = None)

In [14]:
print('************ CountVectorizer ITALIAN-MEXICAN ************')
X = df.loc[:, df.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df['Cuisine'] #just the ids in a list

************ CountVectorizer ITALIAN-MEXICAN ************
(14276, 3976)


- 1. ITALIAN MEXICAN CUISINES CLASSIFIERS (COUNTVECTORIZER)

In [15]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([2.5504601 , 2.18405247, 2.21158576, 1.94333792, 1.90349221]), 'score_time': array([0.02898192, 0.02722764, 0.02730107, 0.02709079, 0.02767682]), 'test_accuracy': array([0.96288515, 0.96672504, 0.97162872, 0.96882662, 0.96812609]), 'test_precision_weighted': array([0.96309073, 0.9668767 , 0.97170751, 0.96893784, 0.96833373]), 'test_recall_weighted': array([0.96288515, 0.96672504, 0.97162872, 0.96882662, 0.96812609]), 'test_f1_weighted': array([0.96283275, 0.96668534, 0.97160427, 0.96879444, 0.96808231])}

mean score test accuracy: 0.967638326784304

mean score test precision weighted: 0.9677893006690403

mean score test recall weighted: 0.967638326784304

mean score test f1-measure weighted: 0.9675998230499061
Confusion Matrix: [[7683  155]
 [ 307 6131]]


In [16]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.4891715 , 0.49625278, 0.49789715, 0.5144918 , 0.49009752]), 'score_time': array([0.02744508, 0.02796507, 0.02731681, 0.02739477, 0.02725601]), 'test_accuracy': array([0.96113445, 0.96322242, 0.9614711 , 0.96532399, 0.96497373]), 'test_precision_weighted': array([0.96116578, 0.96322223, 0.9614711 , 0.96531882, 0.96497238]), 'test_recall_weighted': array([0.96113445, 0.96322242, 0.9614711 , 0.96532399, 0.96497373]), 'test_f1_weighted': array([0.96110708, 0.96320787, 0.9614711 , 0.96532033, 0.96496109])}

mean score test accuracy: 0.9632251394428174

mean score test precision weighted: 0.9632300630213381

mean score test recall weighted: 0.9632251394428174

mean score test f1-measure weighted: 0.9632134975948101
Confusion Matrix: [[7597  241]
 [ 284 6154]]


In [17]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision-Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision-Tree mean scores****

scores: {'fit_time': array([11.39658332, 33.20916319, 10.85297585, 11.07879996, 16.97399926]), 'score_time': array([0.02721238, 0.02717781, 0.02638102, 0.02665997, 0.02704692]), 'test_accuracy': array([0.93487395, 0.94535902, 0.93134851, 0.93870403, 0.94781086]), 'test_precision_weighted': array([0.93497916, 0.94541877, 0.9313223 , 0.93869318, 0.94793324]), 'test_recall_weighted': array([0.93487395, 0.94535902, 0.93134851, 0.93870403, 0.94781086]), 'test_f1_weighted': array([0.93490394, 0.94537721, 0.93131879, 0.93869756, 0.9477437 ])}

mean score test accuracy: 0.9396192732784883

mean score test precision weighted: 0.9396693301209688

mean score test recall weighted: 0.9396192732784883

mean score test f1-measure weighted: 0.9396082410941424
Confusion Matrix: [[7420  418]
 [ 436 6002]]


In [18]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([4.22745609, 4.12844706, 4.11732197, 4.16310835, 4.35632157]), 'score_time': array([173.40004349, 168.35602403, 170.35519505, 172.46086407,
       171.29579973]), 'test_accuracy': array([0.88830532, 0.88406305, 0.88231173, 0.88721541, 0.87845884]), 'test_precision_weighted': array([0.88823164, 0.88608528, 0.88566209, 0.89111778, 0.884777  ]), 'test_recall_weighted': array([0.88830532, 0.88406305, 0.88231173, 0.88721541, 0.87845884]), 'test_f1_weighted': array([0.8881997 , 0.88333008, 0.88132628, 0.8861979 , 0.87695044])}

mean score test accuracy: 0.884070871781287

mean score test precision weighted: 0.8871747597288895

mean score test recall weighted: 0.884070871781287

mean score test f1-measure weighted: 0.8832008807972722
Confusion Matrix: [[7345  493]
 [1162 5276]]


In [19]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****

scores: {'fit_time': array([0.46336746, 0.48547864, 0.46542311, 0.45993423, 0.46202064]), 'score_time': array([0.04326558, 0.02981758, 0.0301919 , 0.02966714, 0.02948952]), 'test_accuracy': array([0.96358543, 0.96462347, 0.96497373, 0.96672504, 0.96532399]), 'test_precision_weighted': array([0.96392113, 0.96502008, 0.96547506, 0.96698809, 0.96576285]), 'test_recall_weighted': array([0.96358543, 0.96462347, 0.96497373, 0.96672504, 0.96532399]), 'test_f1_weighted': array([0.96352096, 0.96455609, 0.96489876, 0.96667345, 0.96525427])}

mean score test accuracy: 0.9650463337699353

mean score test precision weighted: 0.9654334437437194

mean score test recall weighted: 0.9650463337699353

mean score test f1-measure weighted: 0.9649807059388958
Confusion Matrix: [[7704  134]
 [ 365 6073]]


In [20]:
print('************ TfidfVectorizer ITALIAN-MEXICAN ************')
X = df2.loc[:, df2.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df2['Cuisine'] #just the ids in a list

************ TfidfVectorizer ITALIAN-MEXICAN ************
(14276, 2006)


- 2. ITALIAN MEXICAN CUISINES CLASSIFIERS (TFIDFVECTORIZER)

In [21]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([0.52974987, 0.52992105, 0.51307178, 0.4936018 , 0.5006001 ]), 'score_time': array([0.01318645, 0.00974274, 0.00974846, 0.00980425, 0.00980759]), 'test_accuracy': array([0.99264706, 0.99369527, 0.99194396, 0.99579685, 0.99334501]), 'test_precision_weighted': array([0.99274423, 0.99376687, 0.99206049, 0.99582877, 0.99342468]), 'test_recall_weighted': array([0.99264706, 0.99369527, 0.99194396, 0.99579685, 0.99334501]), 'test_f1_weighted': array([0.9926413 , 0.99369109, 0.99193702, 0.99579502, 0.9933403 ])}

mean score test accuracy: 0.993485628927578

mean score test precision weighted: 0.9935650103027747

mean score test recall weighted: 0.993485628927578

mean score test f1-measure weighted: 0.9934809442666724
Confusion Matrix: [[7838    0]
 [  93 6345]]


In [22]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.19516706, 0.19173741, 0.19179201, 0.19114351, 0.19552565]), 'score_time': array([0.0107379 , 0.00990653, 0.00981736, 0.00994205, 0.00989962]), 'test_accuracy': array([0.99614846, 0.99649737, 0.9943958 , 0.9971979 , 0.99579685]), 'test_precision_weighted': array([0.99617529, 0.99651958, 0.99445244, 0.9972056 , 0.99582877]), 'test_recall_weighted': array([0.99614846, 0.99649737, 0.9943958 , 0.9971979 , 0.99579685]), 'test_f1_weighted': array([0.99614693, 0.99649612, 0.99439252, 0.9971973 , 0.99579502])}

mean score test accuracy: 0.9960072750641412

mean score test precision weighted: 0.9960363379540353

mean score test recall weighted: 0.9960072750641412

mean score test f1-measure weighted: 0.9960055765255399
Confusion Matrix: [[7837    1]
 [  56 6382]]


In [23]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision-Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision-Tree mean scores****

scores: {'fit_time': array([2.93528199, 2.91533303, 2.83061743, 2.92146993, 2.83942223]), 'score_time': array([0.01470017, 0.01477718, 0.01456809, 0.01467276, 0.01457882]), 'test_accuracy': array([0.99894958, 0.99824869, 0.99754816, 0.99894921, 0.99859895]), 'test_precision_weighted': array([0.99895159, 0.99825055, 0.99755907, 0.99895122, 0.99860251]), 'test_recall_weighted': array([0.99894958, 0.99824869, 0.99754816, 0.99894921, 0.99859895]), 'test_f1_weighted': array([0.99894947, 0.9982485 , 0.99754755, 0.9989491 , 0.99859875])}

mean score test accuracy: 0.9984589177177

mean score test precision weighted: 0.9984629859026336

mean score test recall weighted: 0.9984589177177

mean score test f1-measure weighted: 0.9984586754890283
Confusion Matrix: [[7837    1]
 [  19 6419]]


In [24]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([2.26060057, 2.25794148, 2.25653839, 2.29463649, 2.26201057]), 'score_time': array([85.24381018, 85.42045975, 85.69957519, 85.78246546, 85.19833612]), 'test_accuracy': array([0.93277311, 0.92924694, 0.9352014 , 0.94500876, 0.93309982]), 'test_precision_weighted': array([0.94010695, 0.93732614, 0.94204372, 0.95001376, 0.94036414]), 'test_recall_weighted': array([0.93277311, 0.92924694, 0.9352014 , 0.94500876, 0.93309982]), 'test_f1_weighted': array([0.93200522, 0.9283789 , 0.93450048, 0.94453364, 0.9323391 ])}

mean score test accuracy: 0.9350660053863928

mean score test precision weighted: 0.9419709424977883

mean score test recall weighted: 0.9350660053863928

mean score test f1-measure weighted: 0.9343514707413195
Confusion Matrix: [[7838    0]
 [ 927 5511]]


In [25]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****
scores: {'fit_time': array([0.10076237, 0.09514427, 0.09485245, 0.09402418, 0.09432292]), 'score_time': array([0.01139355, 0.01068044, 0.01070833, 0.01060581, 0.01093888]), 'test_accuracy': array([0.9772409 , 0.97478109, 0.97688266, 0.9765324 , 0.97618214]), 'test_precision_weighted': array([0.9781468 , 0.97588893, 0.97781698, 0.97749407, 0.97717212]), 'test_recall_weighted': array([0.9772409 , 0.97478109, 0.97688266, 0.9765324 , 0.97618214]), 'test_f1_weighted': array([0.9771774 , 0.9747017 , 0.97681713, 0.97646426, 0.97611174])}

mean score test accuracy: 0.9763238360142656

mean score test precision weighted: 0.9773037810783135

mean score test recall weighted: 0.9763238360142656

mean score test f1-measure weighted: 0.9762544433551351
Confusion Matrix: [[7838    0]
 [ 338 6100]]


- 3. MOROCCAN FILIPINO BRITISH CUISINES CLASSIFIERS (COUNTVECTORIZER)

In [26]:
print('********* CountVectorizer MOROCCAN, FILIPINO, BRITISH *********')
X = df_.loc[:, df_.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df_['Cuisine'] #just the ids in a list

********* CountVectorizer MOROCCAN, FILIPINO, BRITISH *********
(2380, 2074)


In [27]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****

scores: {'fit_time': array([0.27862191, 0.24550509, 0.25401402, 0.25037909, 0.2556076 ]), 'score_time': array([0.00531888, 0.00494123, 0.00494361, 0.00491691, 0.00494671]), 'test_accuracy': array([0.93277311, 0.91386555, 0.92857143, 0.94117647, 0.92436975]), 'test_precision_weighted': array([0.93375299, 0.91415375, 0.93111993, 0.94164264, 0.92566397]), 'test_recall_weighted': array([0.93277311, 0.91386555, 0.92857143, 0.94117647, 0.92436975]), 'test_f1_weighted': array([0.93290052, 0.91388955, 0.92893002, 0.94121051, 0.92471029])}

mean score test accuracy: 0.9281512605042017

mean score test precision weighted: 0.9292666553555075

mean score test recall weighted: 0.9281512605042017

mean score test f1-measure weighted: 0.9283281782038854
Confusion Matrix: [[767  15  39]
 [ 17 688  50]
 [ 19  31 754]]


In [28]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****

scores: {'fit_time': array([0.05639935, 0.05193067, 0.05356455, 0.0527761 , 0.05258369]), 'score_time': array([0.00526214, 0.0049367 , 0.00489259, 0.00505567, 0.00506258]), 'test_accuracy': array([0.93487395, 0.91806723, 0.93277311, 0.94117647, 0.92226891]), 'test_precision_weighted': array([0.93557612, 0.91851772, 0.93457682, 0.94150972, 0.92328605]), 'test_recall_weighted': array([0.93487395, 0.91806723, 0.93277311, 0.94117647, 0.92226891]), 'test_f1_weighted': array([0.93500387, 0.91805646, 0.93309992, 0.94103213, 0.92260506])}

mean score test accuracy: 0.9298319327731093

mean score test precision weighted: 0.9306932854251169

mean score test recall weighted: 0.9298319327731093

mean score test f1-measure weighted: 0.9299594858712522
Confusion Matrix: [[777  13  31]
 [ 16 688  51]
 [ 20  36 748]]


In [29]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****Decision Tree mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Decision Tree mean scores****

scores: {'fit_time': array([0.18254161, 0.1817956 , 0.1562953 , 0.16272569, 0.14570832]), 'score_time': array([0.00505733, 0.0050807 , 0.00495791, 0.00497079, 0.0049839 ]), 'test_accuracy': array([0.83403361, 0.83823529, 0.81932773, 0.85084034, 0.81722689]), 'test_precision_weighted': array([0.83626326, 0.83871892, 0.81944673, 0.8532019 , 0.81896285]), 'test_recall_weighted': array([0.83403361, 0.83823529, 0.81932773, 0.85084034, 0.81722689]), 'test_f1_weighted': array([0.8339944 , 0.83831236, 0.81936487, 0.85094205, 0.81661781])}

mean score test accuracy: 0.8319327731092437

mean score test precision weighted: 0.83331873263702

mean score test recall weighted: 0.8319327731092437

mean score test f1-measure weighted: 0.8318462992499029
Confusion Matrix: [[700  47  74]
 [ 27 622 106]
 [ 69  74 661]]


In [30]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****

scores: {'fit_time': array([0.23669338, 0.23128605, 0.23707008, 0.22060704, 0.22833872]), 'score_time': array([2.30282998, 2.29884911, 2.27051783, 2.33117747, 2.26388431]), 'test_accuracy': array([0.79621849, 0.81512605, 0.79831933, 0.77941176, 0.80252101]), 'test_precision_weighted': array([0.80618893, 0.81544252, 0.7993118 , 0.78054418, 0.80455836]), 'test_recall_weighted': array([0.79621849, 0.81512605, 0.79831933, 0.77941176, 0.80252101]), 'test_f1_weighted': array([0.79741893, 0.8151708 , 0.79822941, 0.77950045, 0.80260625])}

mean score test accuracy: 0.7983193277310925

mean score test precision weighted: 0.8012091576133983

mean score test recall weighted: 0.7983193277310925

mean score test f1-measure weighted: 0.7985851652502454
Confusion Matrix: [[639 108  74]
 [ 53 623  79]
 [ 78  88 638]]


In [31]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****\n')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****

scores: {'fit_time': array([0.04693985, 0.04437375, 0.04418445, 0.04391146, 0.04604006]), 'score_time': array([0.00559258, 0.00527358, 0.00519228, 0.00665784, 0.00526857]), 'test_accuracy': array([0.90336134, 0.90336134, 0.90966387, 0.90756303, 0.91386555]), 'test_precision_weighted': array([0.90708443, 0.90986811, 0.91474134, 0.90976343, 0.91953465]), 'test_recall_weighted': array([0.90336134, 0.90336134, 0.90966387, 0.90756303, 0.91386555]), 'test_f1_weighted': array([0.90387865, 0.90406515, 0.91051916, 0.90788884, 0.91483684])}

mean score test accuracy: 0.907563025210084

mean score test precision weighted: 0.9121983933675686

mean score test recall weighted: 0.907563025210084

mean score test f1-measure weighted: 0.9082377260592935
Confusion Matrix: [[741  10  70]
 [ 15 665  75]
 [ 19  31 754]]
